## BRONZE TO SILVER LAYER


In [1]:
import pandas as pd
from global_paths import TradeHistory_GOLD_FILE_PATH, ProfitLoss_GOLD_FILE_PATH

ModuleNotFoundError: No module named 'global_paths'

### SILVER Layer - Process ProfitLoss History


In [ ]:
# Read the CSV file
df = pd.read_csv(TradeHistory_GOLD_FILE_PATH)

# Filter the DataFrame
df = df[df["side"] == "SELL"]

# Convert datetime to date string
df["datetime"] = pd.to_datetime(df["datetime"])

# Rename
df = df.rename(
    columns={
        "sell_quantity": "quantity",
    }
)

# sort the dataframe by date
df = df.sort_values(by=["segment", "stock_name", "datetime"])

# Save the result as a csv file
df = df[
    [
        "datetime",
        "segment",
        "stock_name",
        "quantity",
        "buy_price",
        "buy_amount",
        "sell_price",
        "sell_amount",
        "profit_loss_amount",
    ]
]
df.to_csv(ProfitLoss_GOLD_FILE_PATH, index=None)
df.info()
print("GOLD Layer csv file for ProfitLoss successfully created at:")
print(ProfitLoss_GOLD_FILE_PATH.resolve())